In [ ]:
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

def similarity(a: str, b: str) -> float:
    emb_1 = encoder.encode(a)
    emb_2 = encoder.encode(b)

    return round(float(encoder.similarity(emb_1, emb_2).item()), 3)

dataset = load_dataset('IlyaGusev/librusec_full', split='train', streaming=True)

with open('All_annotations.json', 'r', encoding='utf-8') as f:
    all_a = json.load(f)

with open('titles.json', 'r', encoding='utf-8') as f:
    needed_titles = json.load(f)
    
annotations_dict = {}
for a in all_a:
    annotations_dict[a['title']] = a
     
SIM_TH = 0.65
texts_dict = {}
needed_titles = set(needed_titles)
encoder = SentenceTransformer('deepvk/USER-bge-m3')

with tqdm(total=496858, desc="Processing records") as pbar:
    for record in dataset:
        title = record.get("title", "")
        authors = record.get("authors", [""])
        lang = record.get("lang", "")

        if title in needed_titles:
            if title not in texts_dict:
                if lang in ['ru', 'rus']:
                    if annotations_dict[title]['author'] in authors:
                        texts_dict[title] = (record.get("sections", ""), authors)
                    else:
                        for author in authors:
                            if similarity(author, annotations_dict[title]['author']) > SIM_TH:
                                texts_dict[title] = (record.get("sections", " "), authors)
                                break

        pbar.set_postfix({
            'found': len(texts_dict),
        })
        pbar.update(1)

        if len(texts_dict) == len(needed_titles):
            break
            
combined = []
for title, value in texts_dict.items():
  combined.append({'title': title, 'author': annotations_dict[title]['author'], 'authors': value[1], 'annotation': annotations_dict[title]['annotation'],'text': value[0], 'categories': annotations_dict[title]['categories']})

Processing records: 100%|██████████| 496858/496858 [4:05:20<00:00, 33.75it/s, found=634]


In [1]:
import sys
sys.path.append('methods')

from methods import Summarisation

C:\Users\dagri\BOOOKSum\utils.py:4: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.1)
  from scipy.special import softmax
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dagri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dagri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
C:\Users\dagri\anaconda3\envs\WIKIbench\Lib\site-packages\ignite\handlers\checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [2]:
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

In [3]:
bench = Summarisation(URL=url, KEY=key, model_name='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta') 

In [7]:
from utils import chunk_text
text = "\n".join(bench.collection[3]['text'])
chunks = chunk_text(text)
print(len(text))
print(len(chunks))

870892
109


In [4]:
from utils import chunk_text
text = bench.collection[0]['text'][0]
chunks = chunk_text(text)
print(len(text))
print(len(chunks))

60664
8


In [5]:
s = await bench.blueprint.run(chunks, initial_word_limit=500)

In [6]:
print(s)

### Краткое содержание

**1. Эволюция отношений Мейзи:**
- Мейзи начинает с недоверия к миссис Уэст, но постепенно испытывает к ней глубокую благодарность, в отличие от враждебного отношения к Бесси, что отражает контраст между поддержкой и мистической угрозой.

**2. История и защита башни Волвердена:**
- Башня символизирует возрождение и амбициозные реставрационные работы, встречая сопротивление у Бесси, хранящей память о разрушении, в то время как миссис Уэст скрывает масштабы, опасаясь суеверий.

**3. Необычные элементы Волвердена:**
- Мейзи восхищается готической криптой с древними символами, Сфинксом и мистическими образами, что пробуждает её интерес к мистическим аспектам и укреплению веры в миссию.

**4. Роль старухи Бесси:**
- Бесси, хранительница мистических знаний, выступает как защитникница священного прошлого, проклиная реставрацию, что усиливает атмосферу тайны и предчувствий, влияя на восприятие Мейзи.

**5. Путешествие и трансформация Мейзи:**
- Путешествие через крипту 

In [ ]:
await bench.evaluate_annotation(bench.collection[0]['text'][0], s)

In [8]:
import time
print('Hierarchical')
for _ in range(3):
    start = time.perf_counter()
    s = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=True)
    end = time.perf_counter()
    count = end - start
    print(f"Time: {count:.2f}")
    break

Hierarchical
Time: 166.78


In [9]:
import time
print('Blueprint')
for _ in range(3):
    start = time.perf_counter()
    s = await bench.blueprint.run(chunks, initial_word_limit=500)
    end = time.perf_counter()
    count = end - start
    print(f"Time: {count:.2f}")

Blueprint
Time: 533.10
Time: 522.88
Time: 537.34
